In [1]:
import torch
import numpy as np
import pickle
from lpne.models import DcsfaNmf
from lpne.plotting import circle_plot
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
#from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as image
import os, sys
import seaborn as sns
import pandas as pd

umc_data_tools_path = "/hpc/home/mk423/Anxiety/Universal-Mouse-Code/"
sys.path.append(umc_data_tools_path)
import umc_data_tools as umc_dt

featureIdx = 5#int(os.environ['SLURM_ARRAY_TASK_ID']) #0-91

flx_data_path = "/work/mk423/Anxiety/final_FLX_{}.pkl"
epm_data_path = "/work/mk423/Anxiety/EPM_{}_dict_May_17.pkl"
oft_data_path = "/work/mk423/Anxiety/OFT_{}_dict_old_features_hand_picked.pkl"

anx_info_dict = "/work/mk423/Anxiety/Anx_Info_Dict.pkl"

feature_list = ["X_psd","X_coh","X_gc"]
old_feature_list = ["X_power_1_2","X_coh_1_2","X_gc_1_2"]
feature_weights = [10,1,1]
RANDOM_STATE = 42
EPSILON = 1e-6
anxInfoDict = pickle.load(open(anx_info_dict,"rb"))
allFeatures = np.hstack([anxInfoDict["powerFeatures"],anxInfoDict["cohFeatures"],anxInfoDict["gcFeatures"]])

currentFeature = np.unique([feature.split(" ")[0] for feature in allFeatures[56*featureIdx:56*(featureIdx+1)]])[0]

saved_model_path = "/work/mk423/Anxiety/MultiTask/singleFeatureModels/"
saved_model_name = "{}_singleFeature_model_{}_components.pt"

results_path = "/hpc/home/mk423/Anxiety/FullDataWork/Validations/"
results_file = results_path + "{}_singleFeature_model_{}_components_results.pkl"

projection_save_path = "/hpc/home/mk423/Anxiety/FullDataWork/Projections/"
plots_path = "/hpc/home/mk423/Anxiety/FullDataWork/Figures/"


/opt/conda/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/hpc/home/mk423/.local/lib/python3.7/site-packages/lpne/pipelines/__init__.py:14: UserWarning: Could not load lpne/pipelines/default_params.yaml!
  warnings.warn("Could not load lpne/pipelines/default_params.yaml!")


In [2]:
with open(flx_data_path.format("train"),"rb") as f:
    train_dict = pickle.load(f)
    
with open(flx_data_path.format("val"),"rb") as f:
    val_dict = pickle.load(f)
    
with open(flx_data_path.format("test"),"rb") as f:
    test_dict = pickle.load(f)
    
flx_X_train = np.hstack([train_dict[feature]*weight for feature,weight in zip(feature_list,feature_weights)])
flx_y_train = train_dict['y_flx']
flx_y_mouse_train = train_dict['y_mouse']
flx_y_hab_train = train_dict['y_hab']
flx_y_time_train = train_dict['y_time']

flx_X_val = np.hstack([val_dict[feature]*weight for feature,weight in zip(feature_list,feature_weights)])
flx_y_val = val_dict['y_flx']
flx_y_mouse_val = val_dict['y_mouse']
flx_y_hab_val = val_dict['y_hab']
flx_y_time_val = val_dict['y_time']

flx_X_test = np.hstack([test_dict[feature]*weight for feature,weight in zip(feature_list,feature_weights)])
flx_y_test = test_dict['y_flx']
flx_y_mouse_test = test_dict['y_mouse']
flx_y_hab_test = test_dict['y_hab']
flx_y_time_test = test_dict['y_time']
flx_y_expDate_test = test_dict['y_expDate']

#Stack Validation and Training Data from timepoints after 30 minutes
flx_X = np.vstack([flx_X_train[flx_y_hab_train==1],flx_X_val[flx_y_hab_val==1]])
flx_y = np.hstack([flx_y_train[flx_y_hab_train==1],flx_y_val[flx_y_hab_val==1]])
flx_y_mouse = np.hstack([flx_y_mouse_train[flx_y_hab_train==1], flx_y_mouse_val[flx_y_hab_val==1]])

with open(oft_data_path.format("train"),'rb') as f:
    train_dict = pickle.load(f)

with open(oft_data_path.format("validation"),'rb') as f:
    val_dict = pickle.load(f)
    
with open(oft_data_path.format("test"),'rb') as f:
    test_dict = pickle.load(f)
    

running_idx = 0
feature_groups = []
for idx,feature in enumerate(old_feature_list):
    f_begin = running_idx
    f_end = f_begin + train_dict[feature].shape[1] 
    if idx == 0:
        f_end = f_end -1
    feature_groups.append((f_begin,f_end))

    running_idx = f_end

NUM_FREQS = 56
NUM_FEATURES = np.hstack([train_dict[feature] for feature in old_feature_list]).shape[1] // NUM_FREQS
scale_vector = np.array([np.arange(1,NUM_FREQS+1) for feature in range(NUM_FEATURES)]).flatten()

#Train Arrays
oft_X_train = np.hstack([train_dict[feature]*weight for feature,weight in zip(old_feature_list,feature_weights)])
oft_y_hc_train = train_dict['y_Homecage'].astype(bool)
oft_y_task_train = ~oft_y_hc_train
oft_y_ROI_train = train_dict['y_ROI']
oft_y_vel_train = train_dict['y_vel']
oft_y_mouse_train = train_dict['y_mouse']
oft_y_time_train = train_dict['y_time']

#Validation Arrays
oft_X_val = np.hstack([val_dict[feature]*weight for feature,weight in zip(old_feature_list,feature_weights)])
oft_y_hc_val = val_dict['y_Homecage'].astype(bool)
oft_y_task_val = ~oft_y_hc_val
oft_y_ROI_val = val_dict['y_ROI']
oft_y_vel_val = val_dict['y_vel']
oft_y_mouse_val = val_dict['y_mouse']
oft_y_time_val = val_dict['y_time']

#Test Arrays
oft_X_test = np.hstack([test_dict[feature]*weight for feature,weight in zip(old_feature_list,feature_weights)])
oft_y_hc_test = test_dict['y_Homecage'].astype(bool)
oft_y_task_test = ~oft_y_hc_test
oft_y_ROI_test = test_dict['y_ROI']
oft_y_vel_test = test_dict['y_vel']
oft_y_mouse_test = test_dict['y_mouse']
oft_y_time_test = test_dict['y_time']
oft_y_expDate_test = test_dict['y_expDate']

oft_X = np.vstack([oft_X_train,oft_X_val])
oft_y_task = np.hstack([oft_y_task_train,oft_y_task_val])
oft_y_mouse = np.hstack([oft_y_mouse_train,oft_y_mouse_val])

with open(epm_data_path.format("train"),"rb") as f:
    epm_train_dict = pickle.load(f)

with open(epm_data_path.format("val"),"rb") as f:
    epm_validation_dict = pickle.load(f)
    
with open(epm_data_path.format("test"),"rb") as f:
    epm_test_dict = pickle.load(f)

#Load the data
NUM_FREQS = 56
NUM_FEATURES = (epm_train_dict["X_power_1_2"].shape[1] + \
                epm_train_dict["X_coh_1_2"].shape[1] + \
                epm_train_dict["X_gc_1_2"].shape[1]) // NUM_FREQS
SCALE_VECTOR = np.array([np.arange(1,57) for feature in range(NUM_FEATURES)]).flatten()

X_train = np.hstack([epm_train_dict[feature]*weight for feature,weight in zip(old_feature_list,feature_weights)])
X_train[X_train<0] = 0
y_train = (epm_train_dict['y_ROI']%2).astype(bool)
y_in_task_mask_train = ~epm_train_dict['y_Homecage'].astype(bool)
y_mouse_train = epm_train_dict['y_mouse']
y_time_train = epm_train_dict['y_time']
train_nan_mask = (epm_train_dict['y_ROI'] > 0)


X_train_task = X_train[np.logical_and(y_in_task_mask_train==1,train_nan_mask)==1]
y_train_task = y_train[np.logical_and(y_in_task_mask_train==1,train_nan_mask)==1]
y_mouse_train_task = y_mouse_train[np.logical_and(y_in_task_mask_train==1,train_nan_mask)==1]
y_time_train_task = y_time_train[np.logical_and(y_in_task_mask_train==1,train_nan_mask)==1]
X_val = np.hstack([epm_validation_dict[feature]*weight for feature,weight in zip(old_feature_list,feature_weights)])


y_val = (epm_validation_dict['y_ROI']%2).astype(bool)
y_in_task_mask_val= ~epm_validation_dict['y_Homecage'].astype(bool)
y_mouse_val = epm_validation_dict['y_mouse']
y_time_val = epm_validation_dict['y_time']
val_nan_mask = (epm_validation_dict['y_ROI'] > 0)

X_val_task = X_val[np.logical_and(y_in_task_mask_val==1,val_nan_mask)==1]
y_val_task = y_val[np.logical_and(y_in_task_mask_val==1,val_nan_mask)==1]
y_mouse_val_task = y_mouse_val[np.logical_and(y_in_task_mask_val==1,val_nan_mask)==1]
y_time_val_task = y_time_val[np.logical_and(y_in_task_mask_val==1,val_nan_mask)==1]

X_test = np.hstack([epm_test_dict[feature]*weight for feature,weight in zip(old_feature_list,feature_weights)])
y_test = (epm_test_dict['y_ROI']%2).astype(bool)
y_in_task_mask_test= ~epm_test_dict['y_Homecage'].astype(bool)
y_mouse_test = epm_test_dict['y_mouse']
y_time_test = epm_test_dict['y_time']
test_nan_mask = (epm_test_dict['y_ROI'] > 0)
epm_y_expDate_test = epm_test_dict['y_expDate']

X_test_task = X_test[np.logical_and(y_in_task_mask_test==1,test_nan_mask)==1]
y_test_task = y_test[np.logical_and(y_in_task_mask_test==1,test_nan_mask)==1]
y_mouse_test_task = y_mouse_test[np.logical_and(y_in_task_mask_test==1,test_nan_mask)==1]
y_time_test_task = y_time_test[np.logical_and(y_in_task_mask_test==1,test_nan_mask)==1]

epm_X = np.vstack([X_train,X_val])
epm_y_task = np.hstack([y_in_task_mask_train,y_in_task_mask_val])
epm_y_mouse = np.hstack([y_mouse_train,y_mouse_val])
epm_y_time = np.hstack([y_time_train,y_time_val])

mt_X_train = np.vstack([flx_X,oft_X,epm_X])[:,56*featureIdx:56*(featureIdx+1)]
mt_y_train = np.hstack([flx_y,oft_y_task,epm_y_task]).reshape(-1,1)
mt_y_mouse_train = np.hstack([flx_y_mouse,oft_y_mouse,epm_y_mouse])

mt_X_test = np.vstack([flx_X_test[flx_y_hab_test==1],oft_X_test,X_test])[:,56*featureIdx:56*(featureIdx+1)]
mt_y_test = np.hstack([flx_y_test[flx_y_hab_test==1],oft_y_task_test,y_in_task_mask_test]).reshape(-1,1)
mt_y_mouse_test = np.hstack([flx_y_mouse_test[flx_y_hab_test==1],oft_y_mouse_test,y_mouse_test])

intercept_mask = OneHotEncoder().fit_transform(mt_y_mouse_train.reshape(-1,1)).todense()
sample_groups = OrdinalEncoder().fit_transform(mt_y_mouse_train.reshape(-1,1))
nmf_groups = np.hstack(
    [
        np.ones(flx_X.shape[0])*0,
        np.ones(oft_X.shape[0]),
        np.ones(epm_X.shape[0])*2
    ]
)

In [4]:
TRAIN = True

for n_components in [30]:
    if TRAIN:
        model = DcsfaNmf(
            n_components=n_components,
            n_sup_networks=3,
            sup_type="sc",
            n_intercepts=intercept_mask.shape[1],
            optim_name="SGD",
            recon_loss="MSE",
            save_folder=saved_model_path,
        )

        model.fit(mt_X_train,
                  mt_y_train,
                  intercept_mask=intercept_mask,
                  y_sample_groups=sample_groups,
                  batch_size=128,
                  lr=1e-3,
                  n_pre_epochs=200,
                  n_epochs=200,
                  nmf_max_iter=20000,
                  nmf_groups=nmf_groups,
                  bootstrap=True,
                  pretrain=True,
                  verbose=True,
                  best_model_name=saved_model_name.format(currentFeature,n_components))

        torch.save(model,saved_model_path + saved_model_name.format(currentFeature,n_components))
    else:
        model = torch.load(saved_model_path + saved_model_name.format(currentFeature,n_components),map_location="cpu")
        model.device="cpu"
    
    break

(55657, 56) (55657, 3)
Pretraining NMF...
Identifying predictive components for supervised network 0


100%|██████████| 30/30 [00:02<00:00, 12.37it/s]


Selecting network: 19 with auc 0.4219296457129364 for sup net 0 using constraint n/a correlation
Identifying predictive components for supervised network 1


100%|██████████| 30/30 [00:02<00:00, 12.44it/s]


Selecting network: 5 with auc 0.424857721552915 for sup net 1 using constraint n/a correlation
Identifying predictive components for supervised network 2


...

Selecting network: 17 with auc 0.4353525179750406 for sup net 2 using constraint n/a correlation


Encoder Pretrain Epoch: 199, Recon Loss: 0.131861: 100%|██████████| 200/200 [08:15<00:00,  2.48s/it]


Beginning Training


Epoch: 199, Current Training MSE: 0.00166854, Current Training by Window ROC-AUC: [0.562198744428256]: 100%|██████████| 200/200 [11:37<00:00,  3.49s/it] 

Saving the last epoch with training MSE: 0.00166854 and AUCs:[0.562198744428256]


In [5]:
model.n_components

30

In [ ]:
plt.figure(figsize=(8,4))
plt.plot(model.recon_hist,label="training recon")
#plt.plot(model.val_recon_hist,label="validation recon")
#plt.axvline(model.best_epoch,color="red",label="best epoch")
plt.title("Training Reconstruction Performance (After Pretraining)")
plt.legend()
plt.xlabel("Epoch")
plt.ylabel("MSE")
plt.show()

plt.figure(figsize=(8,4))
plt.plot(model.pred_hist,label="training auc")
#plt.plot(model.val_pred_hist,label="validation auc")
#plt.axvline(model.best_epoch,color="red",label="best epoch")
plt.title("Training Prediction Performance (After Pretraining)")
plt.xlabel("Epoch")
plt.ylabel("AUC")
plt.show()

In [5]:
flx_test_auc = model.score(flx_X_test[flx_y_hab_test==1,56*featureIdx:56*(featureIdx+1)],
                          flx_y_test[flx_y_hab_test==1].reshape(-1,1),
                            flx_y_mouse_test[flx_y_hab_test==1],
                            return_dict=True)
                                     
flx_mean_test_auc = np.mean([flx_test_auc[key] for key in flx_test_auc.keys()])
flx_stderr_test_auc = np.std([flx_test_auc[key] for key in flx_test_auc.keys()]) / np.sqrt(len(flx_test_auc.keys()))

print("FLX test auc: {:.3f} +/- {:.3f}".format(flx_mean_test_auc,flx_stderr_test_auc))
print(flx_test_auc)

FLX test auc: 0.488 +/- 0.007
{'Mouse69061': [0.5098491649163174], 'Mouse78732': [0.45758164695878], 'Mouse78743': [0.4878354471544715], 'Mouse78745': [0.4806531392676226], 'Mouse78751': [0.5050238659509261], 'Mouse78764': [0.4846497333083797]}


In [6]:
epm_test_auc = model.score(X_test[:,56*featureIdx:56*(featureIdx+1)],
                          y_in_task_mask_test.reshape(-1,1),
                          y_mouse_test,
                          return_dict=True)

epm_mean_test_auc = np.mean([epm_test_auc[key] for key in epm_test_auc.keys()])
epm_stderr_test_auc = np.std([epm_test_auc[key] for key in epm_test_auc.keys()]) / np.sqrt(len(epm_test_auc.keys()))

print("EPM test auc: {:.3f} +/- {:.3f}".format(epm_mean_test_auc,epm_stderr_test_auc))
print(epm_test_auc)

EPM test auc: 0.560 +/- 0.022
{'Mouse0641': [0.5251726121979285], 'Mouse39115': [0.7168694078616669], 'Mouse39121': [0.5881693660695549], 'Mouse39122': [0.6060309319980617], 'Mouse39132': [0.5135451110061408], 'Mouse39135': [0.5969837959249158], 'Mouse6674': [0.5095533498759305], 'Mouse69061': [0.4374369729150863], 'Mouse69071': [0.4842178715654446], 'Mouse69075': [0.6203367172160255], 'Mouse8893': [0.5629821894628158]}


In [7]:
oft_test_auc = model.score(oft_X_test[:,56*featureIdx:56*(featureIdx+1)],
                            oft_y_task_test.reshape(-1,1),
                            oft_y_mouse_test,
                            return_dict=True)

oft_mean_test_auc = np.mean([oft_test_auc[key] for key in oft_test_auc.keys()])
oft_stderr_test_auc = np.std([oft_test_auc[key] for key in oft_test_auc.keys()]) / np.sqrt(len(oft_test_auc.keys()))
print("OFT test auc: {:.3f} +/- {:.3f}".format(oft_mean_test_auc,oft_stderr_test_auc))
print(oft_test_auc)

OFT test auc: 0.660 +/- 0.036
{'Mouse04203': [0.8456728222205403], 'Mouse39115': [0.7618146088957055], 'Mouse39121': [0.7685112606025154], 'Mouse39122': [0.5702672860188851], 'Mouse39132': [0.654817535699537], 'Mouse39135': [0.6145478886595637], 'Mouse69061': [0.46878922078387786], 'Mouse69071': [0.6162384127500407], 'Mouse69075': [0.6397872506234413]}


In [38]:
s = model.project(mt_X_test)
X_recon = model.reconstruct(mt_X_test)+ EPSILON
perc_recon_list = []
for i in range(3):
    X_sup_recon = model.get_comp_recon(torch.Tensor(s).to(model.device),i) + EPSILON
    net_recon_contribution = np.mean(X_sup_recon/X_recon,axis=0) 
    perc_recon_list.append(net_recon_contribution)


In [46]:
results_dict = {
    "flx_auc":flx_test_auc,
    "flx_mean_test_auc":flx_mean_test_auc,
    "flx_stderr_test_auc":flx_stderr_test_auc,
    
    "epm_auc":epm_test_auc,
    "epm_mean_test_auc":epm_mean_test_auc,
    "epm_stderr_test_auc":epm_stderr_test_auc,
    
    "oft_auc":oft_test_auc,
    "oft_mean_test_auc":oft_mean_test_auc,
    "oft_stderr_test_auc":oft_stderr_test_auc,
    
    "n_components":n_components,
    "coefficients":model.classifier[0].weight[0].detach().cpu().numpy(),
    "s_avg":np.mean(s,axis=0)[:3],
    "feature":currentFeature,
}

with open(results_file.format(currentFeature,n_components),"wb") as f:
    pickle.dump(results_dict,f)

tensor([ 0.4240, -0.1858, -0.1920], grad_fn=<SelectBackward0>)

In [43]:
np.mean(s,axis=0)[:3]

array([0.03321023, 0.03113751, 0.02753031], dtype=float32)